<a href="https://colab.research.google.com/github/osvaldoherrera715-cyber/Vehicle-Accident-Prevention/blob/main/Dashcamv1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi


/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [ ]:
%pip install "ultralytics<=8.3.40" supervision roboflow
# prevent ultralytics from tracking your activity
!yolo settings sync=False
import ultralytics
ultralytics.checks()

Ultralytics 8.3.40 🚀 Python-3.12.12 torch-2.8.0+cu126 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 39.6/107.7 GB disk)


In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="omsEY0EeQVA60NK7nabV")
project = rf.workspace("osvaldos-work").project("car-accident-prevention-s8xnw")
version = project.version(2)
dataset = version.download("yolov11")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Car-Accident-Prevention-2 in yolov11:: 100%|██████████| 90/90 [00:00<00:00, 2401.99it/s]


In [ ]:
import os

dataset_path = "/content/Car-Accident-Prevention-2"

print("📂 Dataset structure:\n")
for split in ['train', 'valid', 'test']:
    images_path = os.path.join(dataset_path, 'images', split)
    labels_path = os.path.join(dataset_path, 'labels', split)

    num_images = len(os.listdir(images_path)) if os.path.exists(images_path) else 0
    num_labels = len(os.listdir(labels_path)) if os.path.exists(labels_path) else 0

    print(f"{split.upper()}:")
    print(f"  Images: {num_images}")
    print(f"  Labels: {num_labels}\n")



📂 Dataset structure:

TRAIN:
  Images: 0
  Labels: 0

VALID:
  Images: 0
  Labels: 0

TEST:
  Images: 0
  Labels: 0



In [ ]:
!ls -R /content/Car-Accident-Prevention-2 | head -n 50



/content/Car-Accident-Prevention-2:
data.yaml
README.dataset.txt
README.roboflow.txt
test
train
valid

/content/Car-Accident-Prevention-2/test:
images
labels

/content/Car-Accident-Prevention-2/test/images:
NeighborhoodToHighway_MP4-0031_jpg.rf.e2ee846b048d651b079c340a1a84d456.jpg
NeighborhoodToHighway_MP4-0099_jpg.rf.3a11750e1bdac3252d9647b148cb722d.jpg
NeighborhoodToHighway_MP4-0170_jpg.rf.9c042d5c45d9f89d9b1c96ff471b8d8f.jpg
NeighborhoodToHighway_MP4-0255_jpg.rf.0cc0f797a8a52e0d266b1ee0b7f440d7.jpg

/content/Car-Accident-Prevention-2/test/labels:
NeighborhoodToHighway_MP4-0031_jpg.rf.e2ee846b048d651b079c340a1a84d456.txt
NeighborhoodToHighway_MP4-0099_jpg.rf.3a11750e1bdac3252d9647b148cb722d.txt
NeighborhoodToHighway_MP4-0170_jpg.rf.9c042d5c45d9f89d9b1c96ff471b8d8f.txt
NeighborhoodToHighway_MP4-0255_jpg.rf.0cc0f797a8a52e0d266b1ee0b7f440d7.txt

/content/Car-Accident-Prevention-2/train:
images
labels

/content/Car-Accident-Prevention-2/train/images:
NeighborhoodToHighway_MP4-0000_jpg.rf

In [ ]:
data_yaml = """
train: /content/Car-Accident-Prevention-2/train/images
val: /content/Car-Accident-Prevention-2/valid/images
test: /content/Car-Accident-Prevention-2/test/images

nc: 8
names: ['Hazard Sign', 'Misc. Object', 'Misc. Sign', 'Pedestrian', 'Stop Sign', 'Traffic Cone', 'Vehicle', 'Vehicles Detection']

roboflow:
  workspace: osvaldos-work
  project: car-accident-prevention-s8xnw
  version: 2
  license: MIT
  url: https://universe.roboflow.com/osvaldos-work/car-accident-prevention-s8xnw/dataset/2
"""

with open('/content/Car-Accident-Prevention-2/data.yaml', 'w') as f:
    f.write(data_yaml)

print("✅ data.yaml updated successfully!")


✅ data.yaml updated successfully!


In [ ]:
!pip install -U ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.9 MB/s eta 0:00:00
  Attempting uninstall: ultralytics
    Found existing installation: ultralytics 8.3.40
    Uninstalling ultralytics-8.3.40:
      Successfully uninstalled ultralytics-8.3.40


In [ ]:
from ultralytics import YOLO

# Load the YOLOv11n model (lightweight)
model = YOLO('yolo11n.pt')


WARNING ⚠️ Ultralytics settings reset to default values. This may be due to a possible problem with your settings or a recent ultralytics package update. 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
import os

for split in ['train', 'valid', 'test']:
    img_dir = f'/content/Car-Accident-Prevention-2/{split}/images'
    lbl_dir = f'/content/Car-Accident-Prevention-2/{split}/labels'
    print(f"\n📁 {split.upper()}:")
    print(f"Images found: {len(os.listdir(img_dir))}")
    print(f"Labels found: {len(os.listdir(lbl_dir))}")



📁 TRAIN:
Images found: 27
Labels found: 27

📁 VALID:
Images found: 8
Labels found: 8

📁 TEST:
Images found: 4
Labels found: 4


In [ ]:
model.train(
    data='/content/Car-Accident-Prevention-2/data.yaml',
    epochs=30,
    imgsz=800,
    batch=16
)


Ultralytics 8.3.214 🚀 Python-3.12.12 torch-2.8.0+cu126 CPU (Intel Xeon 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Car-Accident-Prevention-2/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=800, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 4, 6])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e057636e3f0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp /content/runs/detect/train/weights/best.pt /content/drive/MyDrive/


Mounted at /content/drive
